## Packages  

First you need to install all packages which are compatible with the latset version of python. 

Installed packages need to be upgraded.

In [2]:
!pip3 install -r requirements --user

Once packages are installed, you can run the next cell to check whether they work.


In [3]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings
from multiprocessing import Pool
import tensorflow as tf
from tensorflow.python.ops import nn


py_file_location = "..."
os.path.abspath(os.path.join(os.path.dirname(py_file_location), os.path.pardir))

from src.close_policy import *
from src.utils import *
from src.metrics import *
from src.DNN_metrics import *
from src.DNN_model import *
from src.loss_function import *
from src.data_generator import *


Choose GPU if it is available 

In [5]:
print(tf.config.list_physical_devices())
tf.device('GPU:1') 

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


## Channel gain creation

First, you can choose the type of channel to perform the simulations. To do this, simply select the type by entering the number of the desired channel type 

$h_{ij}$ where $i \in$ \{P, S\}, $j \in$ \{P, S\}, \ $i$;



| $h_{ij}$      | $th_{ij}$  |
| ------------- | ---------|
| training set  | test set |


1) Let us first create the channel coefficient for the training set  ($h_{ij}$)

In [36]:
h_PP, h_PR, h_RP, h_SS, h_SR, h_RS, h_SP, h_PS = channel_type() # train data-type


        1. Channel gain with gaussian fading [1]
        2. Channel gain with Anne model [2]
        3. Channel gain with Uniform distribution[3] 
        4. Channel gain with Rician fading [4]
        5. Channel gain with Nakagami fading [5]
        6.Exit/Quit
        
Select channel gain
1
Channel gain created


2) Next, let us do the same thing for the test set

In [37]:
t_h_PP, t_h_PR, t_h_RP, t_h_SS, t_h_SR, t_h_RS, t_h_SP, t_h_PS = channel_type() # test data-type


        1. Channel gain with gaussian fading [1]
        2. Channel gain with Anne model [2]
        3. Channel gain with Uniform distribution[3] 
        4. Channel gain with Rician fading [4]
        5. Channel gain with Nakagami fading [5]
        6.Exit/Quit
        
Select channel gain
1
Channel gain created


3) Once the channel coefficient are created, you can convert them to channel gain 

# Convert channel coefficient to channel gain

In [38]:
g_PP, g_PR, g_RP, g_SS, g_SR, g_RS, g_SP, g_PS =\
np.power(h_PP, 2), np.power(h_PR, 2), np.power(h_RP, 2), np.power(h_SS, 2)\
, np.power(h_SR, 2), np.power(h_RS, 2), np.power(h_SP, 2), np.power(h_PS, 2)

In [39]:
t_g_PP, t_g_PR, t_g_RP, t_g_SS, t_g_SR, t_g_RS, t_g_SP, t_g_PS =\
np.power(t_h_PP, 2), np.power(t_h_PR, 2), np.power(t_h_RP, 2), np.power(t_h_SS, 2)\
, np.power(t_h_SR, 2), np.power(t_h_RS, 2), np.power(t_h_SP, 2), np.power(t_h_PS, 2)

## Data processing

After data creation, it must be filtred to avoide the division by zero on the custom loss function. 
In order to do this, we keep only channel gain greater then a fixed threshold $s$.


In [40]:
g_RP, g_PP, g_SR, g_PR, g_SS, g_RS, g_SP, g_PS = data_filter(g_RP, g_PP, g_SR, g_PR, g_SS, g_RS, g_SP, g_PS)

t_g_RP, t_g_PP, t_g_SR, t_g_PR, t_g_SS, t_g_RS, t_g_SP, t_g_PS = data_filter(t_g_RP, t_g_PP, t_g_SR, t_g_PR, t_g_SS, t_g_RS, t_g_SP, t_g_PS)

## Dataset generation

Let us now make things a bit more interesting and create the dataset then save it on a specific path.

#### <span style="color:red"> Imoprtant: </span>  
#### 1) Data size must be fixed
#### 2) Skip next cells and move to data splitting part, if the dataset is already created







In [3]:
train_size = int(1E6)

test_size = int(2E5) 

In [4]:
# # choose your directory path  
project_sub_path = "dataset"
  
# Parent Directory path
parent_dir = ""
  
# Path
try : 

    path = os.path.join(parent_dir, project_sub_path)
    os.mkdir(path)
    print("Directory '% s' created" % project_sub_path)
except FileExistsError : 
    pass

In [47]:
np.savez(os.path.join(parent_dir,project_sub_path,'dataset_train.npz'),
         g_PP=g_PP,
         g_PS=g_PS,
         g_PR=g_PR,
         g_SP=g_SP,
         g_SS=g_SS,
         g_SR=g_SR,
         g_RP=g_RP,
         g_RS=g_RS
         ) 

print("\n training set generation finished")


 training set generation finished


## Bruteforce (exhaustive search)
to evaluate the generalization capability of our proposed DNN during training; 
a test set containing $2 × 10^5$ channel realisations $g_l$, as well as the optimal resource allocation policy ($\alpha^*, P^*_{R}, P^*_{S}$) obtained by brute force (or exhaustive search) to evaluate the optimality
gap of our predicted solution.


The bruteforce method is applied only for the test set to use it as a benchmark with the DNN predicted results

In [53]:
bf_results = benchmark_generator(np.stack([t_g_RP[:test_size], t_g_PP[:test_size], t_g_SR[:test_size], t_g_PR[:test_size], t_g_SS[:test_size], t_g_RS[:test_size], t_g_SP[:test_size], t_g_PS[:test_size]], axis=1))

np.savez(os.path.join(parent_dir,project_sub_path,'dataset_test.npz'),
         g_PP=t_g_PP[:test_size],
         g_PS=t_g_PS[:test_size],
         g_PR=t_g_PR[:test_size],
         g_SP=t_g_SP[:test_size],
         g_SS=t_g_SS[:test_size],
         g_SR=t_g_SR[:test_size],
         g_RP=t_g_RP[:test_size],
         g_RS=t_g_RS[:test_size],
         R_S=bf_results[:,8], 
         alpha=bf_results[:,9], 
         P_R=bf_results[:,10], 
         P_S=bf_results[:,11]
         ) 

print("\n test set generation finished")


 test set generation finished


## Data splitting
Now, we can load the dataset if it's already created

In [4]:
train_size = int(1E6)

test_size = int(2E5) 

# # choose your directory path  
project_sub_path = "dataset"
  
# Parent Directory path
parent_dir = ""


### Train ###

dataset_train = np.load(os.path.join(parent_dir,project_sub_path,'dataset_train.npz'))

g_PP_tr = dataset_train['g_PP'][:train_size]
g_PS_tr = dataset_train['g_PS'][:train_size]
g_PR_tr = dataset_train['g_PR'][:train_size]
g_SP_tr = dataset_train['g_SP'][:train_size]
g_SS_tr = dataset_train['g_SS'][:train_size]
g_SR_tr = dataset_train['g_SR'][:train_size]
g_RP_tr = dataset_train['g_RP'][:train_size]
g_RS_tr = dataset_train['g_RS'][:train_size]

x_train = np.stack([g_RP_tr, g_PP_tr, g_SR_tr, g_PR_tr, g_SS_tr, g_RS_tr, g_SP_tr, g_PS_tr], axis=1)


### Test ### 

dataset_test = np.load(os.path.join(parent_dir,project_sub_path,'dataset_test.npz'))


g_PP_ts = dataset_test['g_PP'][:test_size]
g_PS_ts = dataset_test['g_PS'][:test_size]
g_PR_ts = dataset_test['g_PR'][:test_size]
g_SP_ts = dataset_test['g_SP'][:test_size]
g_SS_ts = dataset_test['g_SS'][:test_size]
g_SR_ts = dataset_test['g_SR'][:test_size]
g_RP_ts = dataset_test['g_RP'][:test_size]
g_RS_ts = dataset_test['g_RS'][:test_size]

R_S_ts = dataset_test['R_S']
alpha_ts = dataset_test['alpha']#Alpha
P_R_ts = dataset_test['P_R']#p_R
P_S_ts = dataset_test['P_S']#p_S


x_test = np.stack([g_RP_ts, g_PP_ts, g_SR_ts, g_PR_ts, g_SS_ts, g_RS_ts, g_SP_ts, g_PS_ts], axis=1)

y_test = np.stack([R_S_ts, alpha_ts, P_R_ts, P_S_ts], axis=1)


## DNN Training 

In our numerical simulations, we used the
ADAM optimizer [1] to iteratively update the weights of our
DNN. The batch size is set to $4096$, the learning rate to $10^{−4}$;
these values allows the DNN weights optimization to converge
within $1000$ epochs.

[[1]](https://arxiv.org/abs/1412.6980): Kingma, Diederik P., and Jimmy Ba. "Adam: A method for stochastic optimization." arXiv preprint arXiv:1412.6980 (2014).

### Impact of the hyperparameter $\lambda$
In the training stage, we need to investigate precisely the impact of the $\lambda$ on the loss function. To do this, we choose an interval for this hyperparameter, $\lambda \in [10^{-1},...,10^{2}]$.



## System setup

In [5]:
Lambda, tau, P_P = 10**(0.5), 0.25, 10.0

metrics = [opportunistic_rate_DF(Lambda, tau, P_P), outage_DF(Lambda, tau, P_P), delta_DF(Lambda, tau, P_P), delta_out_DF(Lambda, tau, P_P)] 

Epochs = 1000 # Epochs number

BS = 4096 # Batch_size

VS = 0.2 # Validation set
#'10_-1':10**-1, '10_-0.75':10**-0.75, 
LD = {'10_-0.5':10**-0.5, '10_-0.25':10**-0.25, '10_0':10**0, '10_0.25':10**0.25, '10_0.5':10**0.5, '10_0.75':10**0.75, '10_1':10**1, '10_1.25':10**1.25, '10_1.5':10**1.5, '10_1.75':10**1.75, '10_2':10**2}

LR = 10**-4 #{'10_-4':10**-4} # One value or Dict to train the model with several learning rate

root_dir ='DNN'

## DNN training

In [ ]:
for ld_k, ld_v in LD.items():
    
    #Create a new directory to save the network history and weights for each value of lambda 

    lambda_dir = root_dir+'/lambda = '+ld_k+'/weights/'
  
    history_dir = root_dir+'/lambda = '+ld_k+'/history/'

    tf.io.gfile.makedirs(lambda_dir)

    tf.io.gfile.makedirs(history_dir)

    #for lr_k, lr_v in LR.items(): add a loop if browsing learning rate needed

    model = get_model_DF(x_train, loss_DF(ld_v,tau), metrics,'sigmoid', custom_sigmoid, custom_sigmoid, LR) #lr_v
    history = model.fit(x_train, x_train, epochs=Epochs, batch_size=BS, validation_split = VS)

    model.save(lambda_dir+ld_k+'.h5')
    np.save(history_dir+ld_k+'.npy',history.history)

